# Multi-Label evaluator

## Import libraries

In [ ]:
# Basic libraries
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt

# Keras basic libraries
from keras import backend
from keras import Model
from keras.models import load_model
from keras.applications import inception_resnet_v2

## Define Input Parameters and Functions

In [ ]:
# Defining parameters
INPUT_WIDTH = 299
INPUT_HEIGHT = 299
INPUT_SIZE = ( INPUT_WIDTH, INPUT_HEIGHT )
INPUT_SHAPE = ( INPUT_WIDTH, INPUT_HEIGHT, 3 )

# Defining our necessary functions

# Function to decode features from one-hot code
def onehot2features( one_hot ):
    one_hot = np.round( one_hot )
    y = []
    for index, value in enumerate( one_hot ):
        if( value == 1 ):
            y.append( index + 1 )
    return y

# F2-score function for accuracy metric
def fbeta(y_true, y_pred, beta=2):

  y_pred = backend.clip(y_pred, 0, 1)

  tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
  fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
  fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)

  p = tp / (tp + fp + backend.epsilon())
  r = tp / (tp + fn + backend.epsilon())
  
  bb = beta ** 2
  fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
  return fbeta_score

def precision(y_true, y_pred):

  y_pred = backend.clip(y_pred, 0, 1)

  tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
  fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
  fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)

  p = tp / (tp + fp + backend.epsilon())
  
  precision_score = backend.mean(p)
  return precision_score

def recal(y_true, y_pred):

  y_pred = backend.clip(y_pred, 0, 1)

  tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
  fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
  fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)

  r = tp / (tp + fn + backend.epsilon())

  recal_score = backend.mean(r)
  return recal_score

## Loading Model

In [ ]:
dependencies = {
    'fbeta': fbeta, 'precision':precision, 'recal':recal
}
model = load_model('model_classifier.h5', custom_objects=dependencies)

## Labeling Images

In [ ]:
with open('image_labeling_info.json', 'r') as path:
    json_image_info = json.load( path )

labels = []
for image_dict in json_image_info:
    y_pred = inception_resnet_v2.preprocess_input( cv2.resize( cv2.imread( image_dict['file_name'] ), INPUT_SIZE ) )
    y_pred = model.predict( np.expand_dims( y_pred, axis=0 ) )
    y_pred = np.round( y_pred )
    labels.append( onehot2features( np.array( y_pred )[0] ) )

## Saving Labels

In [ ]:
import pickle
with open('labels.pkl','wb') as f:
    pickle.dump( labels, f )